In [1]:
from copy import copy
from dataclasses import dataclass
from typing import Any

def _eval(a):
    try: 
        res = a.result()
        if res == a:
            return copy(a)
    except AttributeError:
        res = a
    return copy(res)

def _validate(a):
    if a not in [0, 1] \
        and not isinstance(a, str) \
        and not isinstance(a, And) \
        and not isinstance(a, Or) \
        and not isinstance(a, Xor) \
        and not isinstance(a, Not):
            raise ValueError("Value must be 0, 1, or an instance of str, And, Or, Xor, or Not.")
    return copy(a)

@dataclass
class And:
    a: Any
    b: Any
    
    def __init__(self, a, b):
        _validate(a)
        _validate(b)
        self.a = a
        self.b = b

    def result(self):
        match self.a, self.b:
            case (0, _) | (_, 0):
                return 0
            case (Not(a), b) | (a, Not(b)) if a == b:
                return 0
            case (1, a) | (a, 1):
                return a
            case _:
                return self

    def __repr__(self):
        return f"And({self.a}, {self.b})"
    def __str__(self):
        r = self.result()
        if r in [0,1] or isinstance(r,str):
            return f"{r}"
        else:
            return f"({self.a}&{self.b})"

@dataclass
class Or:
    a: Any
    b: Any
        
    def __init__(self, a, b):
        _validate(a)
        _validate(b)
        self.a = a
        self.b = b

    def result(self):
        match self.a, self.b:
            case (1, _) | (_, 1):
                return 1
            case (Not(a), b) | (a, Not(b)) if a == b:
                return 1
            case (0, a) | (a, 0):
                return a
            case _:
                return self

    def __repr__(self):
        return f"Or({self.a}, {self.b})"
    
    def __str__(self):
        r = self.result()
        if r in [0,1] or isinstance(r,str):
            return f"{r}"
        else:
            return f"({self.a}|{self.b})"

@dataclass
class Xor:
    a: Any
    b: Any
        
    def __init__(self, a, b):
        self.a = _validate(a)
        self.b = _validate(b)

    def result(self):
        l = _eval(self.a)
        r = _eval(self.b)
        match l, r:
            case (0, a) | (a, 0):
                return a
            case (1, a) | (a, 1):
                return Not(a)
            case (a, b) if a == b:
                return 0
            case (Not(a), b) | (a, Not(b)) if a == b:
                return 1
            case (Xor(a,b),c) | (a,Xor(b,c)):
                if a == b:
                    return c
                elif a == c:
                    return b
                elif b == c:
                    return a
                else:
                    return copy(self)
            case _:
                return copy(self)
    
    def __str__(self):
        return f"({self.a}^{self.b})"

@dataclass
class Not:
    a: Any
        
    def __init__(self, a):
        _validate(a)
        self.a = a

    def result(self):
        if self.a == 0:
            return 1
        if self.a == 1:
            return 0
        if isinstance(self.a, Not):
            return self.a.a
        else:
            return Not(self.a)
        
    def __repr__(self):
        return f"Not({self.a})"
    
    def __str__(self):
        r = self.result()
        if r in [0,1]:
            return f"{r}"
        return f"!{self.a}"

# Example usage:
assert And('a', 0).result()==0
assert And('a', 1).result()=='a'
assert And('a',Not('a')).result()==0
assert Or('a', 1).result()==1
assert Or('a', 0).result()=='a'
assert Or('a',Not('a')).result()==1
assert Xor('a', 1).result()==Not('a')
assert Xor('a', 0).result()=='a'
assert Xor('a',Not('a')).result()==1
assert Xor('a','a').result()==0
assert Xor('a',Xor('a','b')).result()=='b'
assert Not(Not('a')).result()=='a'

In [ ]:
class Bit:
    def __init__(self, a):
        _validate(a)
        self.a = a
    
    def __and__(self,b):
        _validate(b)
        Bit(And(self.a,b))
    
    def __or__(self,b):
        _validate(b)
        Bit(Or(self.a,b))
        
    def __xor__(self,b):
        _validate(b)
        Bit(Xor(self.a,b))
        
    def __not__(self):
        Bit(Not(self.a))


